In [ ]:
# General imports.
import re
import urllib.parse
import zipfile

import ipywidgets as ipw
import numpy as np
from IPython.display import HTML, clear_output, display

# AiiDA imports.
%load_ext aiida
%aiida
from aiida import common

from surfaces_tools.helpers import HART_2_EV

# Local imports.
from surfaces_tools.utils import spm
from surfaces_tools.widgets import series_plotter

from aiida import orm
import os
import ipywidgets as widgets
os.sys.path.append("/home/jovyan/aiida-openbis/")
from aiida_openbis.utils import bisutils

In [ ]:
from pybis import Openbis
def log_in(bisurl='openbis', bisuser='admin', bispasswd='changeit'):
    """Function to login to openBIS."""
    if Openbis(bisurl, verify_certificates=False).is_token_valid():
        session = Openbis(bisurl, verify_certificates=False)
    else:
        Openbis(bisurl, verify_certificates=False).login(bisuser, bispasswd, save_token=True)
        session = Openbis(bisurl, verify_certificates=False)
    return session

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
local_ref_index = None
cp2k_calc = None
orb_calc = None


def load_pk(b):
    global cp2k_calc, orb_calc, new_version, workcalc
    global local_ref_index

    new_version = False
    workcalc = load_node(pk=pk_select.value)
    orb_calc = spm.get_calc_by_label(workcalc, "orb")
    try:
        cp2k_calc = spm.get_calc_by_label(workcalc, "scf_diag")
    except AssertionError:
        try:
            dft_out_params = workcalc.outputs.dft_output_parameters.get_dict()
            new_version = True
        except:
            print("Incorrect pk.")
            return

    geom_info.value = spm.get_slab_calc_info(workcalc.inputs.structure)

    try:
        spm_params = workcalc.inputs.stm_params
    except common.NotExistentAttributeError:
        spm_params = workcalc.inputs.spm_params

    n_homo_inttext.value = max([int(spm_params["--n_homo"]) - 2, 1])
    n_lumo_inttext.value = max([int(spm_params["--n_lumo"]) - 2, 1])

    # Information about the calculation.
    with misc_info:
        clear_output()

    dft_inp_params = dict(workcalc.inputs["dft_params"])
    if not new_version:
        dft_out_params = dict(cp2k_calc.outputs.output_parameters)

    with misc_info:
        if dft_inp_params["uks"]:
            print("UKS multiplicity %d" % dft_inp_params["multiplicity"])
        else:
            print("RKS")

        if "charge" in dft_inp_params:
            print("Charge %d" % dft_inp_params["charge"])
        else:
            print("Charge 0")

        if "init_nel_spin1" in dft_out_params:
            print(
                "Number of alpha (s0) electrons: %d" % dft_out_params["init_nel_spin1"]
            )
            print(
                "Number of beta (s1) electrons:  %d" % dft_out_params["init_nel_spin2"]
            )

        print("Energy [au]: %.6f" % (dft_out_params["energy"]))
        print("Energy [eV]: %.6f" % (dft_out_params["energy"] * HART_2_EV))

        if "--p_tip_ratios" in dict(spm_params):
            p_tip_ratio = spm_params["--p_tip_ratios"]
            print("Tip p-wave contrib: %.2f" % p_tip_ratio)

    # Ionization potential, if it's there.
    with orb_calc.outputs.retrieved.open("_scheduler-stdout.txt") as std_out_file:
        std_out = std_out_file.read()
    matches = re.findall(r"IONIZATION POTENIAL \(eV\): ([\d\.\d]+)", std_out)
    if len(matches) > 0:
        with misc_info:
            print("Ionization potential: %.4f eV" % float(matches[0]))

    # Load data.
    with orb_calc.outputs.retrieved.open("orb.npz", mode="rb") as npz_handle:
        loaded_data = np.load(npz_handle.name, allow_pickle=True)

    s0_orb_general_info = loaded_data["s0_orb_general_info"][()]
    s0_orb_series_info = loaded_data["s0_orb_series_info"]
    s0_orb_series_data = loaded_data["s0_orb_series_data"]

    series_plotter_inst.add_series_collection(
        s0_orb_general_info, s0_orb_series_info, s0_orb_series_data
    )

    ref_index = s0_orb_general_info["homo"]

    if "s1_orb_general_info" in loaded_data.files:
        s1_orb_general_info = loaded_data["s1_orb_general_info"][()]
        s1_orb_series_info = loaded_data["s1_orb_series_info"]
        s1_orb_series_data = loaded_data["s1_orb_series_data"]

        series_plotter_inst.add_series_collection(
            s1_orb_general_info, s1_orb_series_info, s1_orb_series_data
        )

        ref_index = int(0.5 * (ref_index + s1_orb_general_info["homo"]))

    series_plotter_inst.setup_added_collections(workcalc.pk)

    wfn_kit_button.disabled = False

    local_ref_index = np.where(s0_orb_general_info["orb_indexes"] == ref_index)
    local_ref_index = local_ref_index[0][0]


style = {"description_width": "50px"}
layout = {"width": "70%"}

pk_select = ipw.IntText(value=0, description="pk", style=style, layout=layout)

load_pk_btn = ipw.Button(description="Load pk", style=style, layout=layout)
load_pk_btn.on_click(load_pk)

geom_info = ipw.HTML()

display(ipw.HBox([ipw.VBox([pk_select, load_pk_btn]), geom_info]))

misc_info = ipw.Output()
display(misc_info)

# Orbital images

In [ ]:
def selected_orbital_indexes():
    n_homo = n_homo_inttext.value
    n_lumo = n_lumo_inttext.value

    i_start = local_ref_index - n_homo + 1
    i_start = 0 if i_start < 0 else i_start

    i_end = local_ref_index + n_lumo + 1
    i_end = 0 if i_end < 0 else i_end

    return np.arange(i_start, i_end)

In [ ]:
style = {"description_width": "80px"}
layout = {"width": "40%"}

series_plotter_inst = series_plotter.SeriesPlotter(
    select_indexes_function=selected_orbital_indexes, zip_prepend="orbs"
)

n_homo_inttext = ipw.IntText(
    description="num HOMO", min=0, max=100, value=10, style=style, layout=layout
)
n_lumo_inttext = ipw.IntText(
    description="num LUMO", min=0, max=100, value=10, style=style, layout=layout
)

n_orb_select = ipw.HBox(
    [n_homo_inttext, n_lumo_inttext], style=style, layout={"width": "60%"}
)


display(
    series_plotter_inst.selector_widget,
    n_orb_select,
    series_plotter_inst.plot_btn,
    series_plotter_inst.clear_btn,
    series_plotter_inst.plot_output,
)

# Export
**Image zip** exports the currently selected orbital images in png, txt and IGOR pro formats.

**Cube creation kit** creates an archive containing all necessary ingredients to generate the Kohn-Sham orbital cube files with the `cube_from_wfn.py` script available from https://github.com/nanotech-empa/cp2k-spm-tools.

In [ ]:
display(
    ipw.HBox([series_plotter_inst.zip_btn, series_plotter_inst.zip_progress]),
    series_plotter_inst.link_out,
)

In [ ]:
def create_wfn_zip(b):
    wfn_kit_button.disabled = True
    ! mkdir -p tmp
    label = "cube-kit-pk%d" % int(pk_select.value)
    cube_kit_name = label + ".zip"
    # Use 'with' to ensure the zipfile is properly closed after its block
    with zipfile.ZipFile(f"tmp/{cube_kit_name}", "w", zipfile.ZIP_DEFLATED) as zipf:
        if new_version:
            fd = workcalc.outputs["retrieved"]
        else:
            fd = cp2k_calc.outputs["retrieved"]
        for fn in [
            "BASIS_MOLOPT",
            "aiida.inp",
            "aiida.out",
            "aiida.coords.xyz",
            "aiida-RESTART.wfn",
        ]:
            # Use 'with' to ensure the file is properly closed after its block
            with fd.open(fn, "rb") as file_obj:
                zipf.writestr(f"{label}/{fn}", file_obj.read())
    with wfn_kit_output:
        display(
            HTML('<a href="tmp/%s" target="_blank">download zip</a>' % cube_kit_name)
        )


wfn_kit_button = ipw.Button(description="Cube creation kit", disabled=True)
wfn_kit_button.on_click(create_wfn_zip)

wfn_kit_output = ipw.Output()

display(wfn_kit_button, wfn_kit_output)

In [ ]:
def clear_tmp(b):
    ! rm -rf tmp && mkdir tmp
    with series_plotter_inst.link_out:
        clear_output()
    series_plotter_inst.zip_progress.value = 0.0

    with wfn_kit_output:
        clear_output()

    if series_plotter_inst.series is not None:
        series_plotter_inst.zip_btn.disabled = False
        wfn_kit_button.disabled = False


clear_tmp_btn = ipw.Button(description="clear tmp")
clear_tmp_btn.on_click(clear_tmp)
display(clear_tmp_btn)

In [ ]:
# Load the URL after everything is set up.
try:
    url = urllib.parse.urlsplit(jupyter_notebook_url)
    pk_select.value = urllib.parse.parse_qs(url.query)["pk"][0]
    load_pk(0)
except:
    pass

In [ ]:
def get_all_structures_and_geoopts(node):
    """Get all atomistic models that led to the one used in the STM simulation"""
    current_node = node
    all_structures = [node]
    all_geoopts = []
    while current_node is not None:
        if isinstance(current_node, orm.StructureData):
            current_node = current_node.creator
        elif isinstance(current_node, orm.CalcJobNode):
            current_node = current_node.caller
            
        elif isinstance(current_node, orm.WorkChainNode):
            if "GeoOpt" in current_node.label:
                all_geoopts.append(current_node)
                current_node = current_node.inputs.structure
                all_structures.append(current_node)
            else:
                current_node = current_node.caller
    
    return all_structures, all_geoopts

def connect_openbis(b):
    session = log_in(bisurl="openbis", bisuser="admin", bispasswd="123456789")
    
    eln_project_code = "/CARBON_NANOMATERIALS/TRIANGULENE_SPIN_CHAINS"

    # Create Atomistic Models collection
    inventory_project_code = "/MATERIALS/ATOMISTIC_MODELS"
    collection_code = "ATOMISTIC_MODELS_EXP_1"
    collection_not_exists = session.get_collections(project = inventory_project_code, code = collection_code).df.empty

    if collection_not_exists:
        atomistic_models_collection = session.new_collection(project = inventory_project_code, code = collection_code, type = 'COLLECTION')
        atomistic_models_collection.props["$name"] = "Atomistic Models"
        atomistic_models_collection.save()
    else:
        atomistic_models_collection = session.get_collection(code = f"{inventory_project_code}/{collection_code}")
        
    all_experiments = session.get_experiments().df
    
    all_collections_names_codes = []
    
    all_collections = session.get_collections()

    for collection in all_collections:
        name_and_code = (f"{collection.identifier} ({collection.props['$name']})", collection.identifier)
        all_collections_names_codes.append(name_and_code)
    
    experiments_dropdown = widgets.Dropdown(
        options = all_collections_names_codes,
        description = 'Experiment:',
        disabled = False,
    )
    
    export_openbis_btn = widgets.Button(
        description = 'Export to openBIS',
        disabled = False,
        button_style = '', # 'success', 'info', 'warning', 'danger' or ''
        tooltip = 'Click me',
        layout = widgets.Layout(width='auto')
    )
    
    def export_data_to_openbis(b):
        selected_experiment_code = experiments_dropdown.value
        all_experiments_codes = list(all_experiments["identifier"].values)
        experiment_index = all_experiments_codes.index(selected_experiment_code)
        selected_experiment_identifier = all_experiments["permId"].values[experiment_index]
        
        # Get all simulations and atomistic models from openbis
        simulations_openbis = session.get_objects(type = "SIMULATION")
        atomistic_models_openbis = session.get_objects(type = "ATOMISTIC_MODEL")

        # Get STM Simulation Workchain from AiiDA
        stm_workchain_pk = pk_select.value
        stm_workchain_node = load_node(stm_workchain_pk)
        structure_stm = stm_workchain_node.inputs.structure
        structure_stm_pk = structure_stm.pk
        structure_stm_node = load_node(structure_stm_pk)

        # Get Geometry Optimisation Workchain from AiiDA
        all_structures, all_geoopts = get_all_structures_and_geoopts(structure_stm_node)
        all_structures_exist_openbis, all_geoopts_exist_openbis, all_simulations_exist_openbis = [], [], []
        
        # Verify which GeoOpts are already in openBIS
        for geoopt_idx, geoopt in enumerate(all_geoopts):
            geoopt_exists_openbis = False
            for simulation in simulations_openbis:
                if simulation.props.get("wfms-uuid") == geoopt.uuid:
                    all_geoopts_exist_openbis.append([simulation, True])
                    geoopt_exists_openbis = True
                    print(f"GeoOpt {simulation.props.get('wfms-uuid')} already exists.")

            if geoopt_exists_openbis == False:
                all_geoopts_exist_openbis.append([None, False])
        
        # Reverse the lists because in openBIS, one should start by building the parents.
        all_geoopts.reverse()
        all_geoopts_exist_openbis.reverse()
        
        # Verify which structures (atomistic models) are already in openBIS
        for structure_idx, structure in enumerate(all_structures):
            structure_exists_openbis = False
            for atomistic_model in atomistic_models_openbis:
                if atomistic_model.props.get("wfms-uuid") == structure.uuid:
                    all_structures_exist_openbis.append([atomistic_model, True])
                    structure_exists_openbis = True
                    print(f"Structure {atomistic_model.props.get('wfms-uuid')} already exists.")
                
            if structure_exists_openbis == False:
                all_structures_exist_openbis.append([None, False])
        
        # Reverse the lists because in openBIS, one should start by building the parents.
        all_structures.reverse()
        all_structures_exist_openbis.reverse()
        
        # Verify which simulations are already in openBIS
        stm_simulation_exists = False
        for simulation in simulations_openbis:
            if simulation.props.get("wfms-uuid") == stm_workchain_node.uuid:
                all_simulations_exist_openbis.append([simulation, True])
                print(f"STM simulation {simulation.props.get('wfms-uuid')} already exists.")
            
        if stm_simulation_exists == False:
            all_simulations_exist_openbis.append([None, False])
        
        # Build atomistic models (structures in AiiDA) in openBIS
        all_atomistic_models = []
        
        for structure_index, structure in enumerate(all_structures):
            structure_exist_openbis = all_structures_exist_openbis[structure_index]
            
            if structure_exist_openbis[1] == False:
                # Get Structure details from AiiDA
                structure_ase = structure.get_ase()
                # structure_ase.positions # Atoms positions
                # structure_ase.symbols # Atoms Symbols
                # structure_ase.pbc # PBC (X, Y, Z)
                # structure.cell # Cell vectors
                
                # Create Atomistic Model in openBIS
                atomistic_model = session.new_sample(collection = atomistic_models_collection, type='ATOMISTIC_MODEL')
                
                atomistic_model.props['$name'] = "Atomistic Model 1"
                atomistic_model.props['description'] = "A nice atomistic model"
                atomistic_model.props['wfms-uuid'] = structure.uuid
                atomistic_model.props['pbc-x'] = int(structure_ase.pbc[0])
                atomistic_model.props['pbc-y'] = int(structure_ase.pbc[1])
                atomistic_model.props['pbc-z'] = int(structure_ase.pbc[2])
                
                if len(all_structures) > 1 and structure_index == len(all_structures) - 1: # If it is the last of more than one structures, it is optimised.
                    atomistic_model.props['optimised'] = 1
                else:
                    atomistic_model.props['optimised'] = 0
                
                atomistic_model.save()
                
                all_atomistic_models.append(atomistic_model)
            else:
                all_atomistic_models.append(structure_exist_openbis[0])
        
        # Build GeoOpts in openBIS
        all_geoopts_models = []
        
        for geoopt_index, geoopt in enumerate(all_geoopts):
            geoopt_exist_openbis = all_geoopts_exist_openbis[geoopt_index]
            
            if geoopt_exist_openbis[1] == False:
                parent_atomistic_model = all_atomistic_models[geoopt_index]
                geoopt_simulation = session.new_sample(experiment = selected_experiment_identifier, type = 'SIMULATION', parents = [parent_atomistic_model])
                geoopt_simulation.props['$name'] = "GeoOpt Simulation"
                geoopt_simulation.props['description'] = "GeoOpt Simulation"
                geoopt_simulation.props['wfms-uuid'] = geoopt.uuid
                geoopt_simulation.save()
                
                geoopt_model = session.new_sample(experiment = selected_experiment_identifier, type = 'GEO_OPTIMISATION', parents = [geoopt_simulation])
                
                geoopt_model.save()
                
                # Its plus one because there are N+1 geometries for N GeoOpts
                all_atomistic_models[geoopt_index + 1].add_parents(geoopt_model)
                all_atomistic_models[geoopt_index + 1].save()
                
                all_geoopts_models.append(geoopt_model)
            else:
                all_geoopts_models.append(geoopt_exist_openbis[0])
        
        # Build STM simulation in openBIS
        
        if all_simulations_exist_openbis[0][1] == False:
            # Simulated Scanning Tunneling Microscopy
            simulation_model = session.new_sample(experiment = selected_experiment_identifier, type = 'SIMULATION')
            simulation_model.props['$name'] = "Simulation STM"
            simulation_model.props['description'] = "Simulation"
            simulation_model.props['wfms-uuid'] = stm_workchain_node.uuid
            simulation_model.save()

            optimised_atomistic_model = all_atomistic_models[-1]
            stm_simulation_model = session.new_sample(experiment = selected_experiment_identifier, type = 'STM', parents = [simulation_model, optimised_atomistic_model])
            stm_simulation_uuid = stm_workchain_node.uuid
            dft_params = dict(stm_workchain_node.inputs.dft_params)
            
            #TODO: Remove this is the future. Orbitals do not have stm_params
            try:
                stm_params = dict(stm_workchain_node.inputs.stm_params)
                stm_simulation_model.props['emin-J'] = stm_params['--energy_range'][0] * 1.60217663e-19
                stm_simulation_model.props['emax-J'] = stm_params['--energy_range'][1] * 1.60217663e-19
                stm_simulation_model.props['de-J'] = stm_params['--energy_range'][2] * 1.60217663e-19
            except:
                pass
            
            stm_simulation_model.props['$name'] = "Simulated STM"
            stm_simulation_model.props['description'] = "A nice simulated STM"
            stm_simulation_model.save()
            
            stm_simulation_images_zip_filename = series_plotter_inst.create_zip_link_for_openbis()
            
            stm_simulation_images_dataset = session.new_dataset(
                type = "RAW_DATA", 
                files = [stm_simulation_images_zip_filename],
                sample = stm_simulation_model
            )
            stm_simulation_images_dataset.save()

            #TODO: How to do this using Python commands?
            stm_simulation_dataset_filename = "stm_simulation.aiida"
            os.system(f"verdi archive create {stm_simulation_dataset_filename} --no-call-calc-backward --no-call-work-backward --no-create-backward -N {stm_workchain_pk}")

            stm_simulation_dataset = session.new_dataset(
                type = "RAW_DATA", 
                files = [stm_simulation_dataset_filename],
                sample = stm_simulation_model
            )
            stm_simulation_dataset.save()

            # Delete the file after uploading
            os.remove(stm_simulation_dataset_filename)
    
    export_openbis_btn.on_click(export_data_to_openbis)
    display(experiments_dropdown)
    display(export_openbis_btn)
    

openbis_connection_btn = ipw.Button(description="Connect with openBIS")
openbis_connection_btn.on_click(connect_openbis)
display(openbis_connection_btn)